<a href="https://colab.research.google.com/github/Vida0822/MachinLearning_Basic/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 분류 알고리즘 (Classification)

> 럭키백의 생선이 어떤 타깃에 속하는지 확률을 구해주세요! <br>
(생선의 길이, 높이, 무게, 대각선 길이, 대각선 무게가 주어진다)

즉, 특정 클래스에 속할 확률을 구하는 알고리즘이다.

<br>

### K-최근접 이웃 분류기
이웃의 클래스 비율로 각 클래스별 확률을 도출한다. <br>
즉, 검사 대상 이웃들 중 가장 높은 비율을 차지하는 클래스가 나의 클래스이다. <br>

주변 이웃의 클래스 비율 각각 확인 = 이는 곧 나의 클래스별 예측 확률
  -> 내 클래스 예측(최고치)


#### 1) 데이터 준비 (수집, 가공, 전처리)

In [1]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


RAW 데이터를 준비한다(읽어온다).

In [2]:
print(pd.unique(fish['Species'])) # Species 열에서 고유한 값 출력 : unique()
# 데이터 프레임에서 열을 선택하려면 원하는 열을 리스트로 나열한다 => 새로운 데이터프레임으로 반환

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


타킷 데이터에 2개 이상의 클래스가 포함된 문제를 '다중 분류'라고 부른다. <br>
사이킷런에선 편리하게도 문자열로 된 타깃값을 그대로 사용할 수 있다.

<br>

**+) 판다스** <br>

데이터 분석 라이브러리인 판다스의 데이터프레임(dataframe)은 판다스에서 제공하는 2차원 표 형식의 데이터 구조이다. 넘파이처럼 다차원 배열을 다룰 수 있지만 통계와 그래프를 위한 메서드 등 훨씬 풍부한 기능을 제공한다. 넘파이로 상호 변환이 쉽고 사이킷런과 호환성도 좋다.

<br>

In [3]:
# 입력 데이터
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
print(fish_input[:5])

# 타깃 데이터
fish_target = fish['Species'].to_numpy()

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


입력 데이터와 타깃 데이터로 나눈다.

<br>

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state = 42)

훈련 세트와 테스트 세트로 나눈다.

<br>

In [8]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

ss.fit(train_input) # 훈련세트 통계값 반영

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input) # 훈련세트 통계값 tkdyd

데이터 세트를 표준화 전처리

<br>

#### 2) 확률 예측

In [9]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)

# 훈련
kn.fit(train_scaled, train_target)

# 평가
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


이웃 근접 모델을 훈련, 평가한다.

<br>

In [11]:
print(kn.classes_) # 모델에 정렬된 타깃값 저장되어있음

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


※주의 : 문자열 타깃값을 그대로 사이킷런 모델에 전달하면 순서가 자동으로 알파벳 순으로 매겨진다 <br>
즉 모델에 저장된(listing)된 클래스 순서는 알파벳 순이다.

In [14]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


But 친절하게도 순서와 관계없이 타깃값 자체만으로 예측을 출력한다 (not 클래스 순번). <br>
위 코드는 처음 5개 샘플의 타깃값을 예측한 것이다.

<br>

In [16]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5]) # 해당 객체의 클래스별 확률 값 반환
print(np.round(proba, decimals=4)) # 소수점 넷째 자리까지 출력 (반올림)

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


이는 각 객체의 클래스별 확률값을 계산한 것이다. <br>
클래스 순서는 classes_속성의 순서와 같다.

<br>

In [17]:
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

[['Roach' 'Perch' 'Perch']]


고려 대상 이웃 셈플 3개 중 'Roach'가 1개, 'Perch'가 2개이다. <br>
즉 해당 이웃들의 전체 클래스 중 'Roach'가 차지하는 비율은 곧 내 클래스가 'Roach'일 확률이며 <br>
가장 높은 비율(2/3)을 차지하기에 내 클래스는 'Roach'로 예측한다. <br>
같은 원리로, 내 클래스가 'Perch'일 예측확률은 1/3이다.

<br>

👏🏻 최근접 이웃 알고리즘을 사용하라고 지정만 해주면, <br>
입력 데이터의 다양한, 복잡한 특성을 모두 고려하여 이웃과의 관계, 클래스 확률을 도출한다.